In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:

# ---------------- Lectura y guardado de los documentos en un data frame de pandas.


In [26]:
def get_feelings_df(balance_data, lematizacion):
    from gensim.utils import any2unicode as unicode
    from nltk.stem import SnowballStemmer
    from sklearn.utils import resample
    import os, re, sys, pandas, unidecode

    feelings_folder = './entrenamiento de modelos/datos sentimientos'
    feelings_df = pandas.DataFrame(columns=['text', 'lang', 'sentiment'])
    path, subfolders, files_list = list(os.walk(feelings_folder))[0]
    files_list.sort()

    for i in range(len(files_list)):
        sys.stdout.write("\rPreparando df  " + str(round(((i + 1) / (len(files_list))) * 100, 2)) + "%")
        sys.stdout.flush()
        file_name, file_ext = files_list[i].split(".")

        if file_ext == 'csv':
            file_path = path + "/" + file_name + "." + file_ext
            df = pandas.read_csv(file_path, encoding='utf8', dtype=str, engine='python')
            numero_de_archivo = int(file_name.split("_")[0])

            if numero_de_archivo == 1:
                df = df.filter(['airline_sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 2 or numero_de_archivo == 3 or numero_de_archivo == 4:
                df = df.filter(['sentiment', 'text'])
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 5 or numero_de_archivo == 6 or numero_de_archivo == 7:
                df = df.filter(['polarity', 'text'])
                df.columns = ['sentiment', 'text']
                if numero_de_archivo in [5, 6]: df['lang'] = 'es'
                if numero_de_archivo == 7: df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 8:
                df = df.filter(['sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'es'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

    del df
    print("")
    feelings_df.dropna()
    feelings_df.drop_duplicates(subset=['text'], inplace=True)
    feelings_df = feelings_df.loc[feelings_df['sentiment'].isin(['0', '1', '2'])]
    feelings_df = feelings_df.loc[feelings_df['lang'].isin(['es', 'en'])]
    feelings_df.reset_index(drop=True, inplace=True)

    for i, row in feelings_df.iterrows():
        sys.stdout.write("\rNormalizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        feelings_df.at[i, 'text'] = (
            re.sub(' +', ' ', re.sub("http\S+", "", re.sub('\s+', ' ', str(feelings_df.at[i, 'text']))))
        ).strip()

    print("")

    if balance_data == 1:
        feelings_df.dropna()
        feelings_df["Sello"] = 0
        for i, row in feelings_df.iterrows():
            #sys.stdout.write(
             #   "\rCreando sellos de balanceamiento " +
             #   str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2))
             #   + "%"
            #)
            #sys.stdout.flush()
            feelings_df.at[i, 'sello'] = str(feelings_df.at[i, 'lang']) + '_' + str(feelings_df.at[i, 'sentiment'])
        print("\nBalanceando df")
        min_len1 = int(min(feelings_df['sello'].value_counts()))
        df_0 = resample(feelings_df[feelings_df.sello == 'en_0'], replace=False, n_samples=min_len1, random_state=1)
        df_1 = resample(feelings_df[feelings_df.sello == 'en_1'], replace=False, n_samples=min_len1, random_state=1)
        df_2 = resample(feelings_df[feelings_df.sello == 'en_2'], replace=False, n_samples=min_len1, random_state=1)
        df_3 = resample(feelings_df[feelings_df.sello == 'es_0'], replace=False, n_samples=min_len1, random_state=1)
        df_4 = resample(feelings_df[feelings_df.sello == 'es_1'], replace=False, n_samples=min_len1, random_state=1)
        df_5 = resample(feelings_df[feelings_df.sello == 'es_2'], replace=False, n_samples=min_len1, random_state=1)
        feelings_df = pandas.concat([df_0, df_1, df_2, df_3, df_4, df_5])
        feelings_df = feelings_df.filter(['text', 'lang', 'sentiment'])
        feelings_df.reset_index(drop=True, inplace=True)

    if lematizacion == 1:
        stemmer_en = SnowballStemmer('english')
        stemmer_es = SnowballStemmer('spanish')
        for i, row in feelings_df.iterrows():
            #sys.stdout.write("\rLematizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
            #sys.stdout.flush()
            if feelings_df.at[i, 'text'] is str and feelings_df.at[i, 'lang'] == 'es':
                feelings_df.at[i, 'text'] = stemmer_es.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )
            elif feelings_df.at[i, 'text'] is str and feelings_df.at[i, 'lang'] == 'en':
                feelings_df.at[i, 'text'] = stemmer_en.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )
        feelings_df.reset_index(drop=True, inplace=True)
        print("")

    print("Df sentiments entregado\n")
    feelings_df.reset_index(drop=True, inplace=True)
    return feelings_df

In [27]:
df = get_feelings_df(1,1)

Preparando df  100.0%
Normalizando df 0.16%

/home/felipedev/Documentos/entornopruebas/lib/python3.6/site-packages/pandas/core/frame.py:6701: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Normalizando df 9.22%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizando df 24.46%

...

Normalizando df 69.56%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizando df 84.72%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizando df 100.0%

Balanceando df

Df sentiments entregado



In [5]:
print("\nTweets en el dataframe original: " + str(df.shape[0]))


Tweets en el dataframe original: 15564


In [29]:
df.tail()

,text,lang,sentiment
15559,PERIODISMO IRRESPONSABLE Van apareciendo los v...,es,2
15560,"Mañana miércoles 6 de mayo, en Autores en cuar...",es,2
15561,COVID-19 Desescalada comienzo Fase 1 el 11 de ...,es,2
15562,Coronavirus: nuevo comunicado del Comité de Em...,es,2
15563,che en gba no están para flexibilizar nada de ...,es,2


In [31]:
df1 = df

In [32]:
df1

,text,lang,sentiment
0,@SouthwestAir looking forward to the beats mus...,en,0
1,Thanks to @theBeardedCripl and @hannietravels ...,en,0
2,@SouthwestAir great cabin and flight crew this...,en,0
3,Coronavirus and Communities: Activists step up...,en,0
4,@JetBlue Thanks. Still booked our trip 3/13-17...,en,0
5,@JetBlue gr8 #Mint crew on #flight 123 to #LAX...,en,0
6,@SouthwestAir Thank you for the tip!,en,0
7,@southwestair SWEET!!! Glad to hear it. I'll k...,en,0
8,Happy #TT to my friends @AmericanAir . Hope th...,en,0
9,@united Thank you. Took care of everything and...,en,0


In [7]:
#----------------- Lexicones
#Lectura de SentiWordNet Obtenido de 
#https://www.nltk.org/_modules/nltk/corpus/reader/sentiwordnet.html

# SentiWordNet[word] = {POS,	ID,	PosScore,	NegScore}
contador = 0
SentiWordNet = dict()
for lines in open('Data Lexicon/SentiWordNet_3.0.0.txt'):
    if lines.startswith('#'):
        continue
    line = lines.split('\t')
    palabra = line[4].split('#')[0]
    if (palabra in SentiWordNet) or (palabra==''):
        continue
    else:
        SentiWordNet[palabra]={'POS': line[0], 'ID': line[1], 'PosScore': line[2], 'NegScore': line[3]}

In [8]:
# AFFIN[word] = sentiment
AFFIN = dict()
for lines in open('Data Lexicon/AFFIN-111.txt'):
    AFFIN[lines.split('\t')[0]]=(lines.split('\t')[1]).split('\n')[0]

In [9]:
tt = TweetTokenizer()
def Lexicon(data):
    addlex = list()
    for frase in data:
        splited = tt.tokenize(frase)
        sum_swn_neg = 0
        sum_swn_pos = 0
        affin = 0
        stnet = 0
        word_stat = 0
        for word in splited:
            if word in SentiWordNet.keys():
                sum_swn_neg += float(SentiWordNet[word]['NegScore'])
                sum_swn_pos += float(SentiWordNet[word]['PosScore'])
            if word in AFFIN.keys():
                affin += float(AFFIN[word])
            
        addlex.append([sum_swn_neg, sum_swn_pos, affin])
    return addlex

In [10]:
def extension(matriz, data):
    extendLex = np.array(Lexicon(data))
    return np.append(matriz, extendLex, 1)

In [33]:
vectorizer = TfidfVectorizer(max_features=3000, min_df=7, max_df=0.8)

In [35]:
# -------- Español.
df_es = df[df1.lang == 'es']
# -------- Ingles.
df_en = df[df1.lang == 'en']

In [37]:
df_es.shape

(7782, 3)

In [38]:
df_en.shape

(7782, 3)

In [39]:
features = vectorizer.fit_transform(df_en['text']).toarray()

In [43]:
labels = df_en['sentiment']

In [44]:
vectorizacionLexicon = extension(features, df_en['text'])

In [15]:
#for i in vectorizacionLexicon: print(i)

In [16]:
data_train, data_test, label_train, label_test = train_test_split(vectorizacionLexicon,labels)

In [102]:
def entrenar_modelos_sentiments_supervisados(modelo_entr,vectorizacionLexicon,labels):
    from nltk.corpus import stopwords
    from sklearn.naive_bayes import GaussianNB,BernoulliNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import classification_report
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    import pandas as pd
    import pickle, os, nltk


    modelos_en = {
        'GBT': AdaBoostClassifier(DecisionTreeClassifier(max_depth=200), n_estimators=10),
        'RF': RandomForestClassifier(n_estimators=100, max_depth=300),
        'NN': MLPClassifier(hidden_layer_sizes=(50, 2), max_iter=100),
        'DT': DecisionTreeClassifier(max_depth=16),
        'NB': GaussianNB()
        
    }


    print("")
    nltk.download('stopwords')

    # ---------------- Asignacion de los modelos y vectorizadores.
    # -------- Español.
    #modelo_es = modelos_es[modelo_entr]
    #vectorizer_es = TfidfVectorizer(use_idf=True, stop_words=stopwords.words('spanish'))
    # -------- Ingles.
    modelo_en = modelos_en[modelo_entr]
    #vectorizer_en = TfidfVectorizer(use_idf=True, stop_words=stopwords.words('english'))

    # ---------------- Lectura y separacion de datos.
    #df = pd.DataFrame(get_feelings_df(df_balanceado, df_lematizado))
    # -------- Español.
    #df_es = df[df.lang == 'es']
    # -------- Ingles.
    #df_en = df[df.lang == 'en']
    #del df
    print("Separacion de datos por idioma terminada")

    # ---------------- Separacion en data y labels de entrenamiento.
    # -------- Español.
    #data_train_es, data_test_es, label_train_es, label_test_es = train_test_split(
    #    df_es['text'], df_es['sentiment'], random_state=1
    #)
    #del df_es
    # -------- Ingles.
    data_train_en, data_test_en, label_train_en, label_test_en = train_test_split(
        vectorizacionLexicon, labels, random_state=1
    )
    #print("Division de datos terminada")
    #del df_en

    # ---------------- vectorizacion de los textos.
    # -------- Español.
    #training_data_es = vectorizer_es.fit_transform(data_train_es)
    #testing_data_es = vectorizer_es.transform(data_test_es)
    #del data_train_es, data_test_es
    #print("Vectorizacion en español terminada")
    # -------- Ingles.
    


    # -------- Ingles.
    ruta_modelo_en = './entrenamiento de modelos/modelos/' + str(type(modelo_en).__name__) + '_Sentiment_en.sav'
    modelo_en.fit(data_train_en, label_train_en)
    pickle.dump(modelo_en, open(ruta_modelo_en, 'wb'))
    print("Modelo de " + str(type(modelo_en).__name__) + " en ingles guardado en " + ruta_modelo_en)


    # -------- Ingles.
    predictions_en = modelo_en.predict(data_test_en)
    print("Predicciones terminadas")

    # -------- Ingles.
    print("\nResultados " + str(type(modelo_en).__name__) + " Ingles:\n")
    print(classification_report(label_test_en, predictions_en))


#### Entrenamiento de modelos NB:

#### Entrenamiento de modelos DT:

In [103]:
entrenar_modelos_sentiments_supervisados('NB',vectorizacionLexicon,labels)


Separacion de datos por idioma terminada
Modelo de GaussianNB en ingles guardado en ./entrenamiento de modelos/modelos/GaussianNB_Sentiment_en.sav


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Predicciones terminadas

Resultados GaussianNB Ingles:

              precision    recall  f1-score   support

           0       0.55      0.70      0.61       634
           1       0.67      0.70      0.68       675
           2       0.61      0.41      0.49       637

    accuracy                           0.60      1946
   macro avg       0.61      0.60      0.60      1946
weighted avg       0.61      0.60      0.60      1946



In [94]:
entrenar_modelos_sentiments_supervisados('DT',vectorizacionLexicon,labels)


Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de DecisionTreeClassifier en ingles guardado en ./entrenamiento de modelos/modelos/DecisionTreeClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados DecisionTreeClassifier Ingles:

              precision    recall  f1-score   support

           0       0.67      0.64      0.66       634
           1       0.67      0.74      0.71       675
           2       0.64      0.59      0.62       637

    accuracy                           0.66      1946
   macro avg       0.66      0.66      0.66      1946
weighted avg       0.66      0.66      0.66      1946



#### Entrenamiento de modelos RF:

In [84]:
entrenar_modelos_sentiments_supervisados('RF',vectorizacionLexicon,labels)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de RandomForestClassifier en ingles guardado en ./entrenamiento de modelos/modelos/RandomForestClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados RandomForestClassifier Ingles:

              precision    recall  f1-score   support

           0       0.77      0.71      0.74       634
           1       0.76      0.82      0.79       675
           2       0.71      0.71      0.71       637

    accuracy                           0.75      1946
   macro avg       0.75      0.75      0.75      1946
weighted avg       0.75      0.75      0.75      1946



#### Entrenamiento de modelos GBT:

In [104]:
entrenar_modelos_sentiments_supervisados('GBT',vectorizacionLexicon,labels)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de AdaBoostClassifier en ingles guardado en ./entrenamiento de modelos/modelos/AdaBoostClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados AdaBoostClassifier Ingles:

              precision    recall  f1-score   support

           0       0.66      0.67      0.67       634
           1       0.69      0.70      0.69       675
           2       0.62      0.60      0.61       637

    accuracy                           0.66      1946
   macro avg       0.65      0.66      0.66      1946
weighted avg       0.66      0.66      0.66      1946



#### Entrenamiento de modelos NN:

In [86]:
entrenar_modelos_sentiments_supervisados('NN',vectorizacionLexicon,labels)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de MLPClassifier en ingles guardado en ./entrenamiento de modelos/modelos/MLPClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados MLPClassifier Ingles:

              precision    recall  f1-score   support

           0       0.70      0.70      0.70       634
           1       0.77      0.67      0.72       675
           2       0.62      0.69      0.65       637

    accuracy                           0.69      1946
   macro avg       0.69      0.69      0.69      1946
weighted avg       0.69      0.69      0.69      1946



/home/felipedev/Documentos/entornopruebas/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
